# 2 - Redes neuronales residuales (ResNets)

**Sumario**

1. Redes neuronales residuales (ResNet)
2. Transferencia de aprendizaje sobre ResNet

## 2.1 - Redes neuronales residuales (ResNet)

### 2.1.1 - Importación de librerías

### 2.1.2 - Definición del bloque residual

### 2.1.3 - Definición de la red de neuronas

### 2.1.4 - Otros tipos de bloques ResNet

## 2.2 - Transferencia de aprendizaje sobre ResNet

### 2.2.1 - Importación de librerías

### 2.2.2 - Creación de la red